# weibo 크롤링

## 크롤링 조건 설정

In [25]:
# 페이지 스크롤 다운 횟수 설정(숫자가 클수록 많은 페이지 크롤링)
num_scrolldown = 6
# 날짜 설정 : "설정 날짜" 포함, 이후 날짜 데이터 출력됨.
start_date = '2022-01-01'

In [26]:
from bs4 import BeautifulSoup as bs
import time
import random
from selenium import webdriver
import pandas as pd
from datetime import datetime
import getpass
import chromedriver_autoinstaller as ca

## 페이지 로그인

In [27]:
# 사이트 로그인
ca.install(True)
driver = webdriver.Chrome()
driver.get("https://weibo.com/login.php")
time.sleep(5)
driver.implicitly_wait(15)

# 창 최대화
driver.maximize_window()
time.sleep(1)

In [28]:
# 로그인 정보
id = input("ID를 입력하세요 : ")
password = getpass.getpass("password(비밀번호)를 입력하세요 : ")

element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
time.sleep(1)
element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')

element_id.clear()
element_password.clear()

element_id.send_keys(id)
time.sleep( random.uniform(0.5, 1) )
element_password.send_keys(password)
time.sleep( random.uniform(0.5, 1) )

##로그인버튼 클릭
driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()
time.sleep(5)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_9440/3677500735.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
C:\Users\kbjung\AppData\Local\Temp/ipykernel_9440/3677500735.py:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
C:\Users\kbjung\AppData\Local\Temp/ipykernel_9440/3677500735.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()


## 크롤링할 페이지 접속

In [29]:
# 페이지 접속
url = 'https://weibo.com/p/1001066011831795/home?from=page_100106&mod=TAB#place'
# webdriver 로드
driver.get(url)
time.sleep(5)
driver.implicitly_wait(5)

## 페이지 스크롤 다운

In [30]:
# 페이지 스크롤 다운
for i in range(num_scrolldown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep( random.uniform(1,3))
    driver.implicitly_wait(5)

In [31]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
print("타이틀 : ", soup.title.text)

타이틀 :  京畿道官博的微博_微博


## 데이터 크롤링

### 타이틀 수집

In [32]:
# 타이틀 수집
# 타이틀 첫번째 마침표까지만 가져오기
raw_contents = soup.find_all('div', class_='WB_text W_f14')
content_list = []
num = len(raw_contents)
for i in range(num):
    content = raw_contents[i].text.strip().split('。')[0]
    content_list.append(content)
    
print('원본 게시글 수 : ', len(raw_contents))
print('게시글 수 : ', len(content_list))

원본 게시글 수 :  45
게시글 수 :  45


### 조회수 수집

In [33]:
# 조회수 수집
raw_number_view = soup.find_all('i', class_='S_txt2')
number_view_list = []
for i in range(len(raw_number_view)):
    temp = raw_number_view[i].attrs['title']
    number_view = int(temp[9:-1])
    number_view_list.append(number_view)
    
print('조회수 리스트 원소수 : ', len(number_view_list))

조회수 리스트 원소수 :  45


### 좋아요, 공유수, 댓글 수 수집

In [34]:
# 정보 수집
raw_info = soup.find_all('span', 'line S_line1')

In [35]:
# 좋아요 수 수집
number_like_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 1
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    number_like = int(temp)
    number_like_list.append(number_like)
    
print('좋아요 리스트 원소수 : ', len(number_like_list))

좋아요 리스트 원소수 :  45


In [36]:
# 공유 수 수집
number_share_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 3
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '转发' in temp:
        number_share_list.append(0)
    else:
        number_like = int(temp)
        number_share_list.append(number_like)
        
print('공유수 리스트 원소수 : ', len(number_share_list))

공유수 리스트 원소수 :  45


In [37]:
# 댓글수 수집
number_comment_list = []
num_range_like = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range_like):
    num = ( i + 1 ) * 4 - 2
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '评论' in temp:
        number_comment_list.append(0)
    else:
        number_like = int(temp)
        number_comment_list.append(number_like)
        
print('댓글수 리스트 원소수 : ', len(number_comment_list))

댓글수 리스트 원소수 :  45


### 게시글 날짜 수집

In [38]:
# 게시글 날짜 수집
date_list = []
raw_date = soup.find_all('div', 'WB_from S_txt2')
for i in range(len(raw_date)):
    temp = raw_date[i].find('a', 'S_txt2')
    date = temp.attrs['title'][:10]
    date_list.append(date)
    
print('날짜 리스트 원소수 : ', len(date_list))

날짜 리스트 원소수 :  45


### 팔로워 수 수집

In [39]:
# 팔로워 수 수집
number_follower_list = []
number_of_follower = soup.find_all('strong', 'W_f16')
num = int(number_of_follower[1].text)
number_follower_list.append(num)
for i in range(len(date_list)-1):
    number_follower_list.append(0)
    
print('팔로워 수 리스트 원소수 : ', len(number_follower_list))

팔로워 수 리스트 원소수 :  45


### 게시글 링크 수집

In [40]:
# 링크 수집
link_list = []
raw_links = soup.find_all('div', 'WB_from S_txt2')
for link in raw_links:
    temp = 'https://weibo.com' + link.find('a' ,'S_txt2').attrs['href']
    link_list.append(temp)
    
print("링크 리스트 원소수 : ", len(link_list))

링크 리스트 원소수 :  45


## 크롤링 데이터 확인

In [41]:
print('원본 게시글 수 : ', len(raw_contents))
print('게시글 수 : ', len(content_list))
print('조회수 리스트 원소수 : ', len(number_view_list))
print('좋아요 리스트 원소수 : ', len(number_like_list))
print('공유수 리스트 원소수 : ', len(number_share_list))
print('댓글수 리스트 원소수 : ', len(number_comment_list))
print('날짜 리스트 원소수 : ', len(date_list))
print('팔로워 수 리스트 원소수 : ', len(number_follower_list))
print("링크 리스트 원소수 : ", len(link_list))

원본 게시글 수 :  45
게시글 수 :  45
조회수 리스트 원소수 :  45
좋아요 리스트 원소수 :  45
공유수 리스트 원소수 :  45
댓글수 리스트 원소수 :  45
날짜 리스트 원소수 :  45
팔로워 수 리스트 원소수 :  45
링크 리스트 원소수 :  45


In [42]:
# chromedriver 종료
driver.quit()

## 데이터 셋 생성

In [43]:
# 데이터 프레임 생성
df = {'날짜':date_list, '제목':content_list, '조회수':number_view_list, '좋아요_수':number_like_list, '공유수':number_share_list, '댓글수':number_comment_list, '팔로워_수':number_follower_list, '링크':link_list}
data = pd.DataFrame(df)
data

날짜                                                 제목    조회수  \
0   2022-05-06  京畿道，举办时隔3年回归的线下游戏秀”PlayX4”时隔3年在线下举办的活动,在首都圈最大规...   3235   
1   2022-05-04  京畿道高阳市热点，宜家高阳店以低廉的价格提供功能性及优质品质的空间 ，🎶宜家高阳店是为很多人...   3602   
2   2022-05-02  题目: 为培养未来人才的京畿道创意科学-培养中心！内容: 知道京畿道坡州和杨坪最近最火的地方...   4860   
3   2022-04-29  京畿道华城，特色大型展厅咖啡厅，Royal X Club京畿道华城，你想要一个独特的体验吗？...   5634   
4   2022-04-27        京畿道骊州，公仔圣地"Sentoy博物馆"位于京畿道骊州市的公仔圣地Sentoy博物馆   6688   
5   2022-04-27  京畿道 道歌(‘在京畿道休息吧’)Ji Hun Kim 作词 / Onong project...  11009   
6   2022-04-25  京畿道坡州阅读综合文化空间,智慧森林汇集珍贵书籍的公共图书馆，智慧之林世界唯一的图书文化城市...   9462   
7   2022-04-22  让我们成为青少年主人公吧！ 一山Gogo乐园复古风的旱冰场！仅在京畿道可见的Gogo乐园，位...   9249   
8   2022-04-20      充满回忆，具有复古感性的杨平青春博物馆你可以感受到过去的复古感在杨平青春博物馆享受复古风吧   9782   
9   2022-04-18  适合春天去的杨州张旭镇美术馆位于扬州的市立张旭镇美术馆，雕塑公园内不仅有展览公园也是非常适合...  16978   
10  2022-04-15             京畿道代表庆典， 军浦杜鹃花节每年四月，在京畿道军埔市杜鹃花乐园会有很多游客  13553   
11  2022-04-13  在亚洲三大设计大奖上获奖的工厂型特色咖啡厅京畿道 坡州市的代表人气场所的大型一体工厂型咖啡厅...  12759   
12  2022-04-11          韩国巧克力研究所博物馆介绍和体验消息随着巧克力产业的发展，巧克力制作师成为专业职业  16055   
13  2022-04-08                        杨平两水里韩国观光100选中经常被选的京畿道杨平两水里  13561   
14  2022-04-06  韩国有名的铁板烧烤店，高阳市大型铁板烧烤喜欢探访美食店的明星们喜欢的美食店，炭烧铁板烧烤位于...  14632   
15  2022-04-04                   利川市特色咖啡厅西蒙斯露台京畿道利川市有着世界级床品牌西蒙斯工厂  15204   
16  2022-04-01  坡州市游戏云,从废弃的设施迅速崛起为新的乐园京畿道坡州市中央生态公园作为拥有湖泊公园和草坪广...  14792   
17  2022-03-30                           京畿道始兴市是平均居民年龄为38岁以下的年轻城市  20777   
18  2022-03-28  说到韩国食品就会想起的炸鸡不仅有油炸，还有调味炸鸡，有排骨味，有蒜味，粉末味等以多样的制作方...  15474   
19  2022-03-25  "面包巡礼"知道韩国MZ一代们的流行语吗？就是找有名的面包店，而且发在SNS的"面包巡礼"由...  15739   
20  2022-03-23                      走在邻近都市的安静的山路，体验世界文化遗产是非常珍贵的体验  17845   
21  2022-03-21                            京畿道金浦市是自然和都市共存的和平生态旅游城市  17014   
22  2022-03-19                                       现在快到了温暖的三月中旬  18884   
23  2022-03-17                               京畿道坡州市是与北朝鲜开城市最邻近的城市  16805   
24  2022-03-15                                          寒冬已过，暖春到来  16057   
25  2022-03-12      华城八景， 济扶落潮和体验神奇海路 济扶岛这是京畿道华城市不可或缺的8个旅游景点之一的岛屿  17595   
26  2022-03-10    走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花园  15191   
27  2022-03-08  可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...  15420   
28  2022-03-04  欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...  14896   
29  2022-03-02    京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地方  18055   
30  2022-02-28  京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...  16269   
31  2022-02-24                         历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线  21754   
32  2022-02-22       世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市  15509   
33  2022-02-18                       京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区  16764   
34  2022-02-15  以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...  19423   
35  2022-02-10  有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...  18576   
36  2022-02-08                                   介绍一下京畿道安山市家族旅行路线  19206   
37  2022-02-06           保存着历史的自豪的城市富川，大韩民国历史旅行路线京畿道富川市是具有独特个性的城市  19839   
38  2022-02-04                       这里是被得天独厚的自然环境包围的美丽的城市，京畿道果川市  18903   
39  2022-02-01  虎年大吉大吉大利新年蒙福#京畿道##朝鲜半岛##京畿道旅游##虎年大吉##大吉大利##新年蒙...  18378   
40  2022-01-27                              介绍一下京畿道观光名地，幸州山城与历史公园  21264   
41  2022-01-26               拥有很多世界文化遗产的美丽的水原华城2021年在华城举办了‘媒体艺术秀’  19191   
42  2022-01-24                   主题公园，都去过哪里呢介绍一下位于京畿道的国内最大的汽车主题公园  19957   
43  2022-01-20  🎹 风乐响起来!! 🎺🎺在京畿道的博物馆都去过哪里？@@(学习历史最好的地方👍👍)👉京畿道博...  19824   
44  2022-01-17  京畿道平泽市，2021平泽世界文化周成功落下帷幕🎉连接世界的文化散策城市 🏃在京畿道平泽市举...  18353   

    좋아요_수  공유수  댓글수   팔로워_수                                                 링크  
0       3    1    1  541473  https://weibo.com/6011831795/LrLaAm1ZN?from=pa...  
1       2    0    0       0  https://weibo.com/6011831795/LrsTH5KJ4?from=pa...  
2       3    0    0       0  https://weibo.com/6011831795/LraB2EhaB?from=pa...  
3       2    0    0       0  https://weibo.com/6011831795/LqFZB8nxO?from=pa...  
4       2    0    0       0  https://weibo.com/6011831795/LqnKM022S?from=pa...  
5       6    1    0       0  

## 데이터 셋. 날짜로 슬라이싱

In [44]:
# 데이터 셋 날짜로 슬라이싱
slicing_data = data[data['날짜'] >= start_date]
slicing_data

날짜                                                 제목    조회수  \
0   2022-05-06  京畿道，举办时隔3年回归的线下游戏秀”PlayX4”时隔3年在线下举办的活动,在首都圈最大规...   3235   
1   2022-05-04  京畿道高阳市热点，宜家高阳店以低廉的价格提供功能性及优质品质的空间 ，🎶宜家高阳店是为很多人...   3602   
2   2022-05-02  题目: 为培养未来人才的京畿道创意科学-培养中心！内容: 知道京畿道坡州和杨坪最近最火的地方...   4860   
3   2022-04-29  京畿道华城，特色大型展厅咖啡厅，Royal X Club京畿道华城，你想要一个独特的体验吗？...   5634   
4   2022-04-27        京畿道骊州，公仔圣地"Sentoy博物馆"位于京畿道骊州市的公仔圣地Sentoy博物馆   6688   
5   2022-04-27  京畿道 道歌(‘在京畿道休息吧’)Ji Hun Kim 作词 / Onong project...  11009   
6   2022-04-25  京畿道坡州阅读综合文化空间,智慧森林汇集珍贵书籍的公共图书馆，智慧之林世界唯一的图书文化城市...   9462   
7   2022-04-22  让我们成为青少年主人公吧！ 一山Gogo乐园复古风的旱冰场！仅在京畿道可见的Gogo乐园，位...   9249   
8   2022-04-20      充满回忆，具有复古感性的杨平青春博物馆你可以感受到过去的复古感在杨平青春博物馆享受复古风吧   9782   
9   2022-04-18  适合春天去的杨州张旭镇美术馆位于扬州的市立张旭镇美术馆，雕塑公园内不仅有展览公园也是非常适合...  16978   
10  2022-04-15             京畿道代表庆典， 军浦杜鹃花节每年四月，在京畿道军埔市杜鹃花乐园会有很多游客  13553   
11  2022-04-13  在亚洲三大设计大奖上获奖的工厂型特色咖啡厅京畿道 坡州市的代表人气场所的大型一体工厂型咖啡厅...  12759   
12  2022-04-11          韩国巧克力研究所博物馆介绍和体验消息随着巧克力产业的发展，巧克力制作师成为专业职业  16055   
13  2022-04-08                        杨平两水里韩国观光100选中经常被选的京畿道杨平两水里  13561   
14  2022-04-06  韩国有名的铁板烧烤店，高阳市大型铁板烧烤喜欢探访美食店的明星们喜欢的美食店，炭烧铁板烧烤位于...  14632   
15  2022-04-04                   利川市特色咖啡厅西蒙斯露台京畿道利川市有着世界级床品牌西蒙斯工厂  15204   
16  2022-04-01  坡州市游戏云,从废弃的设施迅速崛起为新的乐园京畿道坡州市中央生态公园作为拥有湖泊公园和草坪广...  14792   
17  2022-03-30                           京畿道始兴市是平均居民年龄为38岁以下的年轻城市  20777   
18  2022-03-28  说到韩国食品就会想起的炸鸡不仅有油炸，还有调味炸鸡，有排骨味，有蒜味，粉末味等以多样的制作方...  15474   
19  2022-03-25  "面包巡礼"知道韩国MZ一代们的流行语吗？就是找有名的面包店，而且发在SNS的"面包巡礼"由...  15739   
20  2022-03-23                      走在邻近都市的安静的山路，体验世界文化遗产是非常珍贵的体验  17845   
21  2022-03-21                            京畿道金浦市是自然和都市共存的和平生态旅游城市  17014   
22  2022-03-19                                       现在快到了温暖的三月中旬  18884   
23  2022-03-17                               京畿道坡州市是与北朝鲜开城市最邻近的城市  16805   
24  2022-03-15                                          寒冬已过，暖春到来  16057   
25  2022-03-12      华城八景， 济扶落潮和体验神奇海路 济扶岛这是京畿道华城市不可或缺的8个旅游景点之一的岛屿  17595   
26  2022-03-10    走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花园  15191   
27  2022-03-08  可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...  15420   
28  2022-03-04  欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...  14896   
29  2022-03-02    京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地方  18055   
30  2022-02-28  京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...  16269   
31  2022-02-24                         历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线  21754   
32  2022-02-22       世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市  15509   
33  2022-02-18                       京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区  16764   
34  2022-02-15  以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...  19423   
35  2022-02-10  有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...  18576   
36  2022-02-08                                   介绍一下京畿道安山市家族旅行路线  19206   
37  2022-02-06           保存着历史的自豪的城市富川，大韩民国历史旅行路线京畿道富川市是具有独特个性的城市  19839   
38  2022-02-04                       这里是被得天独厚的自然环境包围的美丽的城市，京畿道果川市  18903   
39  2022-02-01  虎年大吉大吉大利新年蒙福#京畿道##朝鲜半岛##京畿道旅游##虎年大吉##大吉大利##新年蒙...  18378   
40  2022-01-27                              介绍一下京畿道观光名地，幸州山城与历史公园  21264   
41  2022-01-26               拥有很多世界文化遗产的美丽的水原华城2021年在华城举办了‘媒体艺术秀’  19191   
42  2022-01-24                   主题公园，都去过哪里呢介绍一下位于京畿道的国内最大的汽车主题公园  19957   
43  2022-01-20  🎹 风乐响起来!! 🎺🎺在京畿道的博物馆都去过哪里？@@(学习历史最好的地方👍👍)👉京畿道博...  19824   
44  2022-01-17  京畿道平泽市，2021平泽世界文化周成功落下帷幕🎉连接世界的文化散策城市 🏃在京畿道平泽市举...  18353   

    좋아요_수  공유수  댓글수   팔로워_수                                                 링크  
0       3    1    1  541473  https://weibo.com/6011831795/LrLaAm1ZN?from=pa...  
1       2    0    0       0  https://weibo.com/6011831795/LrsTH5KJ4?from=pa...  
2       3    0    0       0  https://weibo.com/6011831795/LraB2EhaB?from=pa...  
3       2    0    0       0  https://weibo.com/6011831795/LqFZB8nxO?from=pa...  
4       2    0    0       0  https://weibo.com/6011831795/LqnKM022S?from=pa...  
5       6    1    0       0  

# 데이터 셋. 인덱스 내림차순 정렬

In [45]:
# 인덱스 내림차순 정렬
final_data = slicing_data.sort_index(ascending=False)
final_data

날짜                                                 제목    조회수  \
44  2022-01-17  京畿道平泽市，2021平泽世界文化周成功落下帷幕🎉连接世界的文化散策城市 🏃在京畿道平泽市举...  18353   
43  2022-01-20  🎹 风乐响起来!! 🎺🎺在京畿道的博物馆都去过哪里？@@(学习历史最好的地方👍👍)👉京畿道博...  19824   
42  2022-01-24                   主题公园，都去过哪里呢介绍一下位于京畿道的国内最大的汽车主题公园  19957   
41  2022-01-26               拥有很多世界文化遗产的美丽的水原华城2021年在华城举办了‘媒体艺术秀’  19191   
40  2022-01-27                              介绍一下京畿道观光名地，幸州山城与历史公园  21264   
39  2022-02-01  虎年大吉大吉大利新年蒙福#京畿道##朝鲜半岛##京畿道旅游##虎年大吉##大吉大利##新年蒙...  18378   
38  2022-02-04                       这里是被得天独厚的自然环境包围的美丽的城市，京畿道果川市  18903   
37  2022-02-06           保存着历史的自豪的城市富川，大韩民国历史旅行路线京畿道富川市是具有独特个性的城市  19839   
36  2022-02-08                                   介绍一下京畿道安山市家族旅行路线  19206   
35  2022-02-10  有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...  18576   
34  2022-02-15  以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...  19423   
33  2022-02-18                       京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区  16764   
32  2022-02-22       世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市  15509   
31  2022-02-24                         历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线  21754   
30  2022-02-28  京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...  16269   
29  2022-03-02    京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地方  18055   
28  2022-03-04  欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...  14896   
27  2022-03-08  可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...  15420   
26  2022-03-10    走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花园  15191   
25  2022-03-12      华城八景， 济扶落潮和体验神奇海路 济扶岛这是京畿道华城市不可或缺的8个旅游景点之一的岛屿  17595   
24  2022-03-15                                          寒冬已过，暖春到来  16057   
23  2022-03-17                               京畿道坡州市是与北朝鲜开城市最邻近的城市  16805   
22  2022-03-19                                       现在快到了温暖的三月中旬  18884   
21  2022-03-21                            京畿道金浦市是自然和都市共存的和平生态旅游城市  17014   
20  2022-03-23                      走在邻近都市的安静的山路，体验世界文化遗产是非常珍贵的体验  17845   
19  2022-03-25  "面包巡礼"知道韩国MZ一代们的流行语吗？就是找有名的面包店，而且发在SNS的"面包巡礼"由...  15739   
18  2022-03-28  说到韩国食品就会想起的炸鸡不仅有油炸，还有调味炸鸡，有排骨味，有蒜味，粉末味等以多样的制作方...  15474   
17  2022-03-30                           京畿道始兴市是平均居民年龄为38岁以下的年轻城市  20777   
16  2022-04-01  坡州市游戏云,从废弃的设施迅速崛起为新的乐园京畿道坡州市中央生态公园作为拥有湖泊公园和草坪广...  14792   
15  2022-04-04                   利川市特色咖啡厅西蒙斯露台京畿道利川市有着世界级床品牌西蒙斯工厂  15204   
14  2022-04-06  韩国有名的铁板烧烤店，高阳市大型铁板烧烤喜欢探访美食店的明星们喜欢的美食店，炭烧铁板烧烤位于...  14632   
13  2022-04-08                        杨平两水里韩国观光100选中经常被选的京畿道杨平两水里  13561   
12  2022-04-11          韩国巧克力研究所博物馆介绍和体验消息随着巧克力产业的发展，巧克力制作师成为专业职业  16055   
11  2022-04-13  在亚洲三大设计大奖上获奖的工厂型特色咖啡厅京畿道 坡州市的代表人气场所的大型一体工厂型咖啡厅...  12759   
10  2022-04-15             京畿道代表庆典， 军浦杜鹃花节每年四月，在京畿道军埔市杜鹃花乐园会有很多游客  13553   
9   2022-04-18  适合春天去的杨州张旭镇美术馆位于扬州的市立张旭镇美术馆，雕塑公园内不仅有展览公园也是非常适合...  16978   
8   2022-04-20      充满回忆，具有复古感性的杨平青春博物馆你可以感受到过去的复古感在杨平青春博物馆享受复古风吧   9782   
7   2022-04-22  让我们成为青少年主人公吧！ 一山Gogo乐园复古风的旱冰场！仅在京畿道可见的Gogo乐园，位...   9249   
6   2022-04-25  京畿道坡州阅读综合文化空间,智慧森林汇集珍贵书籍的公共图书馆，智慧之林世界唯一的图书文化城市...   9462   
5   2022-04-27  京畿道 道歌(‘在京畿道休息吧’)Ji Hun Kim 作词 / Onong project...  11009   
4   2022-04-27        京畿道骊州，公仔圣地"Sentoy博物馆"位于京畿道骊州市的公仔圣地Sentoy博物馆   6688   
3   2022-04-29  京畿道华城，特色大型展厅咖啡厅，Royal X Club京畿道华城，你想要一个独特的体验吗？...   5634   
2   2022-05-02  题目: 为培养未来人才的京畿道创意科学-培养中心！内容: 知道京畿道坡州和杨坪最近最火的地方...   4860   
1   2022-05-04  京畿道高阳市热点，宜家高阳店以低廉的价格提供功能性及优质品质的空间 ，🎶宜家高阳店是为很多人...   3602   
0   2022-05-06  京畿道，举办时隔3年回归的线下游戏秀”PlayX4”时隔3年在线下举办的活动,在首都圈最大规...   3235   

    좋아요_수  공유수  댓글수   팔로워_수                                                 링크  
44      5    1    1       0  https://weibo.com/6011831795/Lbck79XJz?from=pa...  
43      3    1    0       0  https://weibo.com/6011831795/LbCqQmNzF?from=pa...  
42      2    0    0       0  https://weibo.com/6011831795/LcdTRzXmm?from=pa...  
41      3    1    1       0  https://weibo.com/6011831795/Lcyygmjkp?from=pa...  
40      1    0    1       0  https://weibo.com/6011831795/LcBRrAZvy?from=pa...  
39      5    0    1       0  

## 엑셀 파일로 출력

In [46]:
# 파일로 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'weibo_' + file_name + '.xlsx'
path = 'crawled_data/weibo/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

weibo_20220509-102339.xlsx 파일 생성 완료


# 코드 마지막